In [57]:
from sklearn.cluster import KMeans
import numpy as np
import os
from sklearn.metrics import accuracy_score

In [58]:
train_dir = "./data/train/"
test_dir = "./data/test/"
out_dir = "./output/"
num_clus = 20
num_exam = 5000

In [24]:
def read_npy(directory):
    iterr=0
    label = []
    files = os.listdir(directory)
    for file_name in files:
        numpy_file = np.load(directory + file_name)
        if iterr==0:
            data = numpy_file
        else:
            data = np.concatenate((data,numpy_file))
        label.append(file_name[:-4])
        iterr+=1
    return data,label

In [47]:
train_data,file_labels = read_npy(train_dir)
test_data,_ = read_npy(test_dir)
actual_label = []
for i in range(len(train_data)):
    actual_label.append(i // num_exam)

In [54]:
def predicted(predicted_cluster,cluster_to_label):
    predicted_label = []
    for i in range(len(predicted_cluster)):
        predicted_label.append(cluster_to_label[predicted_cluster[i]])
    return predicted_label

In [40]:
max_iterr = [10,20,30,40,50,300]
train_accur = []
for num_iter in max_iterr:    
    kmeans = KMeans(n_clusters=num_clus,max_iter=num_iter)
    kmeans.fit(train_data)
    predicted_cluster = kmeans.predict(train_data)
    label_cluster_mat = np.zeros((num_clus,num_clus))
    for i in range(len(predicted_cluster)):
        cluster_no = predicted_cluster[i]
        actual_no = actual_label[i]
        label_cluster_mat[cluster_no][actual_no] +=1
    cluster_to_label = []
    for i in range(num_clus):
        max_labels = np.argmax(label_cluster_mat[i])
        cluster_to_label.append(max_labels)
    predicted_label = predicted(predicted_cluster,cluster_to_label)
    train_accuracy = accuracy_score(actual_label,predicted_label)
    print (num_iter,train_accuracy)
    train_accur.append(train_accuracy)
    test_cluster = kmeans.predict(test_data)
    test_label = predicted(test_cluster,cluster_to_label)
    out_name = "kmeans"+str(num_iter)+".csv"
    f = open(out_dir+out_name,"w")
    f.write("ID,CATEGORY\n")
    for i in range(len(test_label)):
        f.write(str(i)+","+file_labels[test_label[i]]+"\n")    

In [67]:
train_accuracy = accuracy_score(actual_label,predicted_label)
print (1,train_accuracy)
# train_accur.append(train_accuracy)
test_cluster = kmeans.predict(test_data)
test_label = predicted(test_cluster,cluster_to_label)
out_name = "kmeans"+str(1)+".csv"
f = open(out_dir+out_name,"w")
f.write("ID,CATEGORY\n")
for i in range(len(test_label)):
    f.write(str(i)+","+file_labels[test_label[i]]+"\n")

1 0.26718
